In [1]:
from pyspark import SparkContext
import ast
import pandas as pd

In [2]:
sc = SparkContext(appName ="ReviewAnalyzer")

# Перед импортом файлов, они были предварительно залиты в hdfs с помощью команд:
./hdfs dfs -copyFromLocal /home/ubuntu/Desktop/meta_Electronics.json /HW1_2_meta
./hdfs dfs -copyFromLocal /home/ubuntu/Desktop/Electronics_5.json /HW1_2_electronics


In [3]:
# Загрузка и парсинг данных из HDFS
meta_data = sc.textFile("/HW1_2_meta").map(lambda x: ast.literal_eval(x))
ratings_data = sc.textFile("/HW1_2_electronics").map(lambda x: ast.literal_eval(x))


In [4]:
# Вычисление среднего рейтинга для каждого товара
ratings_rdd = ratings_data.map(lambda x: (x['asin'], x['overall'])).groupByKey()
average_ratings = ratings_rdd.mapValues(lambda x: sum(x) / len(x))


In [5]:
# Сопоставление данных с наименованиями товаров
products_rdd = meta_data.map(lambda x: (x['asin'], x.get('title', 'Unknown')))
joined_rdd = average_ratings.join(products_rdd)

In [6]:
# Формирование RDD товаров с рейтингом меньше 3 и вывод топ-10
low_rated_products = joined_rdd.filter(lambda x: x[1][0] < 3)
top_10_low_rated = low_rated_products.takeOrdered(10, key=lambda x: x[1][0])


In [7]:
data = [
    (product[0], product[1][0], product[1][1])
    for product in top_10_low_rated
]
df = pd.DataFrame(data, columns=["asin", "Средний рейтинг", "Наименование товара"])

display(df)

,asin,Средний рейтинг,Наименование товара
0,B000H13L4Y,1.0,ATI TV Wonder 200 PCI Video Card w/PVR Capabil...
1,B00111JODG,1.0,StarTech HDMISPL1HH 1 feet Standard HDMI Cable...
2,B000F1ORW6,1.0,GE 24746 Futura HDTV Ready Antenna
3,B001T9N0R6,1.0,Zeikos 57-in-1 USB 2.0 Flash Memory Card Reade...
4,B001BLN91Y,1.0,Maxtor OneTouch 4 - 1 TB USB 2.0 Desktop Exter...
5,B000NNFS4C,1.0,RCA DRC8335 DVD Recorder &amp; VCR Combo With ...
6,B003KIQTXG,1.0,NEEWER&reg; Photographic Barn Door &amp; Honey...
7,B00000JBIA,1.0,Agfa ePhoto SMILE 0.2MP Digital Camera
8,B0013WI5SS,1.0,Dynex-DX-AP100 Adapter Mini DVI to Mini-DIN
9,B004GGRPGG,1.0,Rapid USB Charger Adapters include: Wall and C...


In [9]:
low_rated_products.saveAsTextFile("/HW1_2_output")

# чтобы скачать файл с hdfs хранилища я использовал команду
./hdfs dfs -copyToLocal /HW1_2_output /home/ubuntu/Desktop/HW1_2_output